# Spatio-temporal analysis of the actin fusion focus component
## obtain aligned trajectories files 


All POI subfolders should be in the same folder, POI folder should contain a '/trajectories' folder with all Cell_ folder containing the trajectories and cytoplasmic intensity from one event

Input POI names in files list, color of channel for h- Myo52 as RefM and for h+ Myo52 as RefP 


In [1]:
#files containing fusion events folder
path= '/mnt/c/Users/Valentine/Documents/Analysis/POI_replicats/' 
files = ['Dni2','Fus1'] #'Prm1', 'Dni1','Exo70', 'Exo84', 'Myo51', 'Cdc8'
#color of h-/h+ spatial marker
RefM= '_B' #'_B' if POI in h+ cell, '_R' if POI in h- cell
RefP= '_R' #'_R' if POI in h+ cell, '_B' if POI in h- cell

import required modules, trajalign from A.picco and define fuction to retrieve data, align in time (cf V.Thomas) and in space (cf A.Picco)


In [2]:
from trajalign.traj import Traj
from trajalign.average import load_directory
import numpy as np
import os

def replicats_nb(POI):
    rep_list = [ folder for folder in os.listdir(path+POI) if 'Rep_' in folder ]
    r= len(rep_list)
    return r

def get_data(POI,rep) :
    directory_rep = os.listdir( path+POI+'/Rep_'+str(rep+1)+'/trajectories') #absolute path of directory to charge data from
    f = [ folder for folder in directory_rep if 'cell_' in folder ] # 'cell_' -> folder nam to list
    return f 

#determin fusion frame based on cytoplasmic fluorescence intensity
def fusion_frame (m_c):
    for x in m_c:
        n=len(x)   # number of frames
        U=np.empty( ((n-2),2) ) # creation of an empty array of n-2 lines and 2 columns
        
            
        for tau in range ((n-2)):  
# construction of the model for frame = x            
            m1 = np.mean(x.f()[0:tau])   
            m2 = np.mean(x.f()[(tau+1):n])
            m = np.array([m1] * tau + [m2]* (n-tau)) # use np.array because python can't substract list
            e = x.f() - m                           # substract theorical model from data
            U[tau,0] = x.frames()[tau]               # first column of matrix = frame number
            U[tau,1] = np.sum(e**2)                  # second column of matrix = sum of square errors 
        pos= np.argmin( U[:,1])              # get position of frame corresponding to mimimum sum of squared errors
        f_f=x.frames()[pos]              #get frame of fusion
        f_f= f_f-1   #correct for diff frame numeration PT/Fiji
        
    return f_f 

#align trajectory such that centermass trajectory from h- cell = origin 
def fusion_axis( t_l ) :
	# input a trajectory list t_l

	# search for the red trajectory, and compute its
	# center of mass
    for t in t_l :
	
		# select the blue trajectory
        if RefM in t.annotations()[ 'file' ] :
			# compute the center of mass of the trajctory
            r = t.center_mass()

	# subtract the center of mass of the red trajectory
	# from all trajectories in the cell. The center of mass
	# of the red trajectory will definethe origin of the x-y
	# plane
    for t in t_l :
		
		# translate all the trajectories accordingly
        t.translate( -r )
		# normalise the flurescence intensities between 0 and 1
        t.norm_f() # normalise between 0 and 1

	# compute the center of mass of the blue trajectory.
	# As the trajectory has been now translated by r, 
	# its center of mass defines the vector of the cell
	# fusion reference frame defined by the two center of
	# mass positions of the Myo52-RFP and Myo52-BFP
    for t in t_l :

		# select the red trajectory
        if RefP in t.annotations()[ 'file' ] :
			# compute the center of mass of the trajctory
            b = t.center_mass()
			# normalise the vector to a unit vector
            v = [ i / np.sqrt( b[ 0 ] ** 2 + b[ 1 ] ** 2 ) for i in b ]

    return t_l , v 


In [3]:
n= len(files) 
Rep= []
for i in range(n):
    r=replicats_nb(files[i])
    Rep.append(r)
folders=[[] for _ in range(n)] #lists for POI rep folders => each subset will contains n subset for each replicat

for i in range(len(Rep)):
    folders[i]=[[] for _ in range(Rep[i])] 

# list all cell_ folders for each POI 

for i in range(n) :
    for r in range(Rep[i]):
        f=get_data(files[i],r)
        folders[i][r]= f 
        

In [4]:
# get trajectories for each fusions sites of each POI
# trajectorie classes => frames, coord = centroid coord; coord_err= [0]mu20 et [1] mu02; f_err= mu11 

ct=0
for f in folders:
    r=1
    print(path+files[ct])
    for rep in f:
        id=0
        print(path+files[ct]+'/Rep_'+str(r)+'/trajectories/')
        for event in rep: 
            m_c = load_directory(path+files[ct] +'/Rep_'+str(r)+ '/trajectories/' + event, pattern= '.csv',  comment_char = '#' ,sep= ',', frames = 0 , f = 1 )
            t_l = load_directory(path+files[ct] + '/Rep_'+str(r)+'/trajectories/' + event , comment_char = '%' , frames = 0 , coord = ( 1 , 2 ) , f = 3 ,coord_err=(11 ,12),f_err = 10 ,coord_unit = 'pxl' )   
            f_f = fusion_frame (m_c)
            for c in m_c:
                centered_c= c.frames()-f_f
                c.input_values('frames',centered_c)
                c.time (3, min) 
                c.norm_f() 
        #center trajectory in time 
            for t in t_l:
                centered_f= t.frames()-f_f     #remove f_f from frame number to center around fusion time (=0)
                t.input_values('frames',centered_f) #replace fram number in trajectories by centered frame number
                scaled_t= t.coord()/ (15.4321)
                t.input_values('coord', scaled_t)   # scale: 15.4321 px/µm, get coordonated as µm
                t.time (3, min)                       # enter time value for each point of the trajectory
            t_l_0 , v = fusion_axis( t_l )

	# rotate the trajectory, so that the fusion axis is 
	# aligned with x positive axis
            angle = np.sign( v[ 0 ] ) * np.arcsin( v[ 1 ] ) - ( np.sign( v[ 0 ] ) - 1 ) * np.pi / 2 
	# add angle offset (we want to see red on the left of the plot, blue on the right)
            angle = angle + np.pi 

	# rotate each trajectory in the trajectory list + save it in new folder 
            os.makedirs(path+files[ct]+'/aligned_trajectories/Rep_'+str(r)+'/cell_%03d/' %id, exist_ok=True)
            print(path+files[ct]+'/aligned_trajectories/Rep_'+str(r)+'/cell_%03d/' %id)
            for t_0 in t_l_0 :
                t_0.rotate( -angle )
                if RefM in t_0.annotations()[ 'file' ] :
                    t_0.save(path+files[ct]+'/aligned_trajectories/Rep_'+str(r)+'/cell_%03d/Traj_M.txt'%id)
                elif RefP in t_0.annotations()[ 'file' ] :
                    t_0.save(path+files[ct]+'/aligned_trajectories/Rep_'+str(r)+'/cell_%03d/Traj_P.txt'%id)
                else :
                    t_0.save(path+files[ct]+'/aligned_trajectories/Rep_'+str(r)+'/cell_%03d/Traj_POI.txt'%id)
        
            id+=1
        r+=1
    ct+=1


/mnt/c/Users/Valentine/Documents/Analysis/POI_replicats/Dni2
/mnt/c/Users/Valentine/Documents/Analysis/POI_replicats/Dni2/Rep_1/trajectories/
1-mCherry_02.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

1-Traj_02_B-01.txt
1-Traj_02_G-01.txt
1-Traj_02_R-01.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/Analysis/POI_replicats/Dni2/aligned_trajectories/Rep_1/cell_000/
1-mCherry_08.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

1-Traj_08_B-01.txt
1-Traj_08_G-01.txt
1-Traj_08_R-01.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/Analysis/POI_replicats/Dni2/aligned_trajectories/Rep_1/cell_001/
2-mCherry_12.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

2-Traj_12_B-03.txt
2-Traj_12_G-0

/mnt/c/Users/Valentine/Documents/Scripts/notebook/jupyterenv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/mnt/c/Users/Valentine/Documents/Scripts/notebook/jupyterenv/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


/mnt/c/Users/Valentine/Documents/Analysis/POI_replicats/Dni2/aligned_trajectories/Rep_1/cell_004/
2-mCherry_16.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

2-Traj_16_B-02.txt
2-Traj_16_G-01.txt
2-Traj_16_R-02.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/Analysis/POI_replicats/Dni2/aligned_trajectories/Rep_1/cell_005/
2-mCherry_17.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

2-Traj_17_B-01.txt
2-Traj_17_G-01.txt
2-Traj_17_R-01.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/Analysis/POI_replicats/Dni2/aligned_trajectories/Rep_1/cell_006/
2-mCherry_18.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

2-Traj_18_B-01.txt
2-Traj_18_G-02.txt
2-Traj_18_R-02.txt

 >> load_directory

/mnt/c/Users/Valentine/Documents/Analysis/POI_replicats/Dni2/aligned_trajectories/Rep_1/cell_031/
3-mCherry_12.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

3-Traj_12_B-01.txt
3-Traj_12_G-01.txt
3-Traj_12_R-01.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/Analysis/POI_replicats/Dni2/aligned_trajectories/Rep_1/cell_032/
3-mCherry_14.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

3-Traj_14_B-03.txt
3-Traj_14_G-02.txt
3-Traj_14_R-04.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/Analysis/POI_replicats/Dni2/aligned_trajectories/Rep_1/cell_033/
3-mCherry_16.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

3-Traj_16_B-01.txt
3-Traj_16_G-03.txt
3-Traj_16_R-02.txt

 >> load_directory


 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/Analysis/POI_replicats/Dni2/aligned_trajectories/Rep_2/cell_012/
4-mCherry_02.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

4-Traj_02_B-01.txt
4-Traj_02_G-01.txt
4-Traj_02_R-01.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/Analysis/POI_replicats/Dni2/aligned_trajectories/Rep_2/cell_013/
4-mCherry_03.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

4-Traj_03_B-01.txt
4-Traj_03_G-01.txt
4-Traj_03_R-01.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/Analysis/POI_replicats/Dni2/aligned_trajectories/Rep_2/cell_014/
4-mCherry_05.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectori

...

3-Traj_09_G-01.txt
3-Traj_09_R-01.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/Analysis/POI_replicats/Fus1/aligned_trajectories/Rep_1/cell_203/
3-mCherry_10.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

3-Traj_10_B-02.txt
3-Traj_10_G-02.txt
3-Traj_10_R-01.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/Analysis/POI_replicats/Fus1/aligned_trajectories/Rep_1/cell_204/
3-mCherry_12.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

3-Traj_12_B-01.txt
3-Traj_12_G-01.txt
3-Traj_12_R-01.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/Analysis/POI_replicats/Fus1/aligned_trajectories/Rep_1/cell_205/
3-mCherry_13.csv

 >> load_directory: The 'intensity_no

1-Traj_17_G-01.txt
1-Traj_17_R-01.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/Analysis/POI_replicats/Fus1/aligned_trajectories/Rep_2/cell_008/
1-mCherry_19.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

1-Traj_19_B-01.txt
1-Traj_19_G-01.txt
1-Traj_19_R-01.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/Analysis/POI_replicats/Fus1/aligned_trajectories/Rep_2/cell_009/
1-mCherry_34.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

1-Traj_34_B-01.txt
1-Traj_34_G-01.txt
1-Traj_34_R-01.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/Analysis/POI_replicats/Fus1/aligned_trajectories/Rep_2/cell_010/
2-mCherry_02.csv

 >> load_directory: The 'intensity_no


 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

2-Traj_44_B-01.txt
2-Traj_44_G-01.txt
2-Traj_44_R-01.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/Analysis/POI_replicats/Fus1/aligned_trajectories/Rep_2/cell_034/
2-mCherry_45.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

2-Traj_45_B-01.txt
2-Traj_45_G-01.txt
2-Traj_45_R-01.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/Analysis/POI_replicats/Fus1/aligned_trajectories/Rep_2/cell_035/
2-mCherry_47.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

2-Traj_47_B-01.txt
2-Traj_47_G-01.txt
2-Traj_47_R-01.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/Analysis

4-Traj_09_R-01.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/Analysis/POI_replicats/Fus1/aligned_trajectories/Rep_2/cell_065/
4-mCherry_11.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

4-Traj_11_B-01.txt
4-Traj_11_G-02.txt
4-Traj_11_R-01.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/Analysis/POI_replicats/Fus1/aligned_trajectories/Rep_2/cell_066/
4-mCherry_14.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

4-Traj_14_B-01.txt
4-Traj_14_G-01.txt
4-Traj_14_R-01.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/Analysis/POI_replicats/Fus1/aligned_trajectories/Rep_2/cell_067/
4-mCherry_15.csv

 >> load_directory: The 'intensity_normalisation' applie

5-Traj_17_G-03.txt
5-Traj_17_R-04.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/Analysis/POI_replicats/Fus1/aligned_trajectories/Rep_2/cell_094/
5-mCherry_18.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

5-Traj_18_B-01.txt
5-Traj_18_G-01.txt
5-Traj_18_R-01.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/Analysis/POI_replicats/Fus1/aligned_trajectories/Rep_2/cell_095/
5-mCherry_19.csv

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

5-Traj_19_B-01.txt
5-Traj_19_G-01.txt
5-Traj_19_R-01.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

/mnt/c/Users/Valentine/Documents/Analysis/POI_replicats/Fus1/aligned_trajectories/Rep_2/cell_096/
5-mCherry_20.csv

 >> load_directory: The 'intensity_no